In [60]:
import numpy as np
import pandas as pd
from tqdm import tqdm
# For GARCH
from arch import arch_model

# For statistical tests
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import rankdata, t, norm

# Copula from sklarpy
from copulae import GaussianCopula, StudentCopula, ClaytonCopula, GumbelCopula

csv_file = 'Diversified_Portfolio_Data_Complete.csv'

# Read CSV, parse 'Date' as a datetime, set it as the index
df_prices = pd.read_csv(csv_file, parse_dates=['Date'], index_col='Date')

# Compute daily (simple) returns, drop the first NaN row
df_returns = df_prices.pct_change().dropna()

# Optionally, you could compute log-returns:
# df_returns = np.log(df_prices).diff().dropna()

# Number of assets is just the number of columns (Tron, Solana, etc.)
assets = df_returns.columns
n_assets = len(assets)

print("Loaded price data with shape:", df_prices.shape)
print("Computed daily returns with shape:", df_returns.shape)

###############################################################################
# HELPER FUNCTIONS FOR GARCH FITTING & DIAGNOSTICS
###############################################################################

def fit_garch_models(returns_series, dist_candidates=('normal', 'StudentsT', 'skewt')):
    """
    Fit multiple GARCH(1,1) models (with different distributions) to a single asset.
    Returns the best model (lowest AIC).
    """
    best_model = None
    best_aic = np.inf
    for dist in dist_candidates:
        try:
            am = arch_model(returns_series, vol='GARCH', p=1, o=0, q=1, dist=dist, rescale=True)
            res = am.fit(disp='off')
            if res.aic < best_aic:
                best_aic = res.aic
                best_model = res
        except Exception as e:
            print(f"Warning: Could not fit dist={dist} -> {e}")
            continue
    return best_model

def ljung_box_tests(std_resids, lags=10):
    """
    Ljung-Box tests on standardized residuals (raw & squared).
    Returns a dict with p-values.
    """
    std_resids = std_resids.dropna()
    # Ljung-Box on raw standardized residuals
    lb_raw = acorr_ljungbox(std_resids, lags=[lags], return_df=True)
    pval_raw = lb_raw['lb_pvalue'].iloc[-1]

    # Ljung-Box on squared standardized residuals
    lb_sq = acorr_ljungbox(std_resids**2, lags=[lags], return_df=True)
    pval_sq = lb_sq['lb_pvalue'].iloc[-1]

    return {
        'LjungBox_pval_raw': pval_raw,
        'LjungBox_pval_sq': pval_sq
    }

def to_uniform(series):
    """
    Convert data to pseudo-uniform [0,1] via rank transformation (empirical CDF).
    """
    ranks = rankdata(series, method='ordinal')
    return (ranks - 0.5) / len(series)


Loaded price data with shape: (1561, 9)
Computed daily returns with shape: (1560, 9)


In [57]:
###############################################################################
# 1) ROLLING WINDOW DEMO FOR GARCH FITTING
###############################################################################

window_size = 500  # Rolling window size

vol_forecasts = pd.DataFrame(index=df_returns.index, columns=df_returns.columns)
std_resids_full = pd.DataFrame(index=df_returns.index, columns=df_returns.columns)
chosen_models = {}

for asset in df_returns.columns:
    for t in tqdm(range(window_size, len(df_returns)), desc=f"Fitting GARCH of asset {asset}"):
        train_data = df_returns[asset].iloc[t - window_size:t]
        
        # Fit best GARCH model
        best_fit = fit_garch_models(train_data.dropna())
        if best_fit is None:
            continue
        
        chosen_models[(asset, t)] = best_fit
        
        # 1-step ahead variance forecast
        forecast_obj = best_fit.forecast(horizon=1, start=train_data.index[-1])
        var_forecast = forecast_obj.variance.iloc[-1]['h.1']
        
        # Store forecasted volatility
        vol_forecasts.loc[df_returns.index[t], asset] = np.sqrt(var_forecast)
        
        # Store in-sample standardized residuals
        insample_std_res = best_fit.resid / best_fit.conditional_volatility
        std_resids_full.loc[:df_returns.index[t-1], asset] = insample_std_res

Fitting GARCH of asset Tron:   0%|          | 0/1060 [00:00<?, ?it/s]

Fitting GARCH of asset Bitcoin:  98%|█████████▊| 1044/1060 [03:58<00:04,  3.34it/s]c:\Users\marko\anaconda3\envs\rllib-torch\lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
Fitting GARCH of asset PolkaDot: 100%|██████████| 1060/1060 [02:44<00:00,  6.43it/s]


In [86]:
# Drop NaN values from standardized residuals before rolling copula fitting
std_resids_full = std_resids_full.dropna()
vol_forecasts = vol_forecasts.dropna()

In [87]:
std_resids_full


,Tron,Solana,Avalanche,Xrp,Bnb,Bitcoin,Doge,Etherium,PolkaDot
Date,,,,,,,,,
2023-08-18,0.08527,-0.663939,0.716214,-0.080372,-0.807827,-1.431468,1.860801,-0.889693,0.853421
2023-08-19,0.758721,0.604592,-0.08011,1.045981,0.171048,-0.066543,0.3338,0.248473,0.053973
2023-08-20,1.269808,-0.286197,0.143924,1.227743,-0.194349,0.029991,-0.038834,0.481145,0.016961
2023-08-21,-0.611494,-0.907221,-2.03214,-0.72302,-1.828229,-0.297831,-1.211036,-0.645457,-0.946365
2023-08-22,0.099808,-1.033179,-0.924795,-0.18647,-0.027659,-0.353534,-0.038691,-1.180846,-0.144435
...,...,...,...,...,...,...,...,...,...
2024-12-25,0.144952,-0.055072,-0.419437,-0.344391,0.185561,0.099591,-0.181111,-0.013333,-0.148757
2024-12-26,-0.880353,-1.203642,-1.258643,-1.956411,-0.611074,-1.164416,-1.339104,-1.187864,-1.406751
2024-12-27,0.879024,-0.640705,-0.391866,-0.115828,0.046609,-0.606194,-0.10954,-0.039281,-0.080757


In [11]:
###############################################################################
# 1) ROLLING/EXPANDING WINDOW DEMO
#    - We'll do a simplified "expanding window" from day=0 to day=t-1 for each t.
###############################################################################

initial_train_size = 800

vol_forecasts = pd.DataFrame(index=df_returns.index, columns=df_returns.columns)
std_resids_full = pd.DataFrame(index=df_returns.index, columns=df_returns.columns)
chosen_models = {}

for asset in df_returns.columns:
    for t in range(initial_train_size, len(df_returns)):
        train_data = df_returns[asset].iloc[:t]
        
        # Fit best GARCH model
        best_fit = fit_garch_models(train_data.dropna())
        if best_fit is None:
            continue
        
        chosen_models[(asset, t)] = best_fit
        
        # 1-step ahead variance forecast
        forecast_obj = best_fit.forecast(horizon=1, start=train_data.index[-1])
        var_forecast = forecast_obj.variance.iloc[-1]['h.1']
        
        # Store forecasted volatility
        vol_forecasts.loc[df_returns.index[t], asset] = np.sqrt(var_forecast)
        
        # Store in-sample standardized residuals
        insample_std_res = best_fit.resid / best_fit.conditional_volatility
        std_resids_full.loc[:df_returns.index[t-1], asset] = insample_std_res


c:\Users\marko\anaconda3\envs\rllib-torch\lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\marko\anaconda3\envs\rllib-torch\lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\marko\anaconda3\envs\rllib-torch\lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\marko\anaconda3\envs\rllib-torch\lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for l

In [88]:
last_day_idx = len(df_returns) - 1  # the final day index
for asset in assets:
    best_fit = chosen_models.get((asset, last_day_idx), None)
    if best_fit:
        # Standardized residuals
        stdres = best_fit.resid / best_fit.conditional_volatility
        
        # Ljung-Box test on residuals
        lb_results = ljung_box_tests(stdres.dropna(), lags=10)
        
        # Print diagnostics (excluding the unavailable 'dist' attribute)
        print(f"[{asset}] AIC={best_fit.aic:.2f}, "
              f"LjungBox pval raw={lb_results['LjungBox_pval_raw']:.3f}, "
              f"LjungBox pval sq={lb_results['LjungBox_pval_sq']:.3f}")
    else:
        print(f"[{asset}] - No final model found.")


[Tron] AIC=-257.84, LjungBox pval raw=0.755, LjungBox pval sq=1.000
[Solana] AIC=604.59, LjungBox pval raw=0.958, LjungBox pval sq=0.463
[Avalanche] AIC=645.75, LjungBox pval raw=0.322, LjungBox pval sq=0.152
[Xrp] AIC=225.27, LjungBox pval raw=0.347, LjungBox pval sq=0.368
[Bnb] AIC=2356.07, LjungBox pval raw=0.923, LjungBox pval sq=0.002
[Bitcoin] AIC=2317.89, LjungBox pval raw=0.658, LjungBox pval sq=0.640
[Doge] AIC=505.31, LjungBox pval raw=0.481, LjungBox pval sq=0.500
[Etherium] AIC=2470.62, LjungBox pval raw=0.830, LjungBox pval sq=0.833
[PolkaDot] AIC=434.10, LjungBox pval raw=0.440, LjungBox pval sq=0.917


In [89]:
print(std_resids_full)

                Tron    Solana Avalanche       Xrp       Bnb   Bitcoin  \
Date                                                                     
2023-08-18   0.08527 -0.663939  0.716214 -0.080372 -0.807827 -1.431468   
2023-08-19  0.758721  0.604592  -0.08011  1.045981  0.171048 -0.066543   
2023-08-20  1.269808 -0.286197  0.143924  1.227743 -0.194349  0.029991   
2023-08-21 -0.611494 -0.907221  -2.03214  -0.72302 -1.828229 -0.297831   
2023-08-22  0.099808 -1.033179 -0.924795  -0.18647 -0.027659 -0.353534   
...              ...       ...       ...       ...       ...       ...   
2024-12-25  0.144952 -0.055072 -0.419437 -0.344391  0.185561  0.099591   
2024-12-26 -0.880353 -1.203642 -1.258643 -1.956411 -0.611074 -1.164416   
2024-12-27  0.879024 -0.640705 -0.391866 -0.115828  0.046609 -0.606194   
2024-12-28 -0.264495  1.293905  0.495143  0.460494  1.274521  0.229937   
2024-12-29   -0.2304 -0.716379  -0.87179 -1.158786 -1.144336 -0.614597   

                Doge  Etherium  Polka

In [97]:
len(copula.params.rho)

36

In [108]:
###############################################################################
# 2) ROLLING WINDOW DEMO FOR COPULA FITTING
###############################################################################

copula_candidates = {
    'Gaussian': GaussianCopula,
    'Student-t': StudentCopula,
    'Clayton': ClaytonCopula,
    'Gumbel': GumbelCopula
}

copula_results = {}
rolling_window_size = 100  # Same rolling window size for copula fitting

for t in tqdm(range(rolling_window_size, len(std_resids_full)), desc="Fitting copulas"):
    rolling_data = std_resids_full.iloc[t - rolling_window_size:t].dropna()

    # Check if enough data exists to fit the copula
    if rolling_data.shape[0] <= rolling_data.shape[1]:  # Not enough rows
        print(f"Skipping window at t={t} due to insufficient data")
        continue

    # Convert to pseudo-observations
    u_data = rolling_data.apply(lambda col: to_uniform(col), axis=0)

    best_copula = None
    best_aic = np.inf

    for copula_name, copula_class in copula_candidates.items():
        try:
            copula = copula_class(dim=u_data.shape[1])
            copula.fit(u_data.values)

            # Compute Negative Log-Likelihood (NLL)
            nll = -copula.pdf(u_data.values).sum()

            # Safely calculate the number of parameters
            if copula_name == 'Student-t':
                num_params = len(copula.params[1]) + 1  # Add 1 for degrees of freedom
            elif copula_name == 'Gaussian':
                num_params = len(copula.params)  # Directly an array
            else:  # Clayton or Gumbel (scalar parameter)
                num_params = 1
                
            # Compute AIC: AIC = 2 * num_params + 2 * NLL
            aic = 2 * num_params + 2 * nll

            if aic < best_aic:
                best_aic = aic
                best_copula = (copula_name, copula)

        except Exception as e:
            print(f"Warning: Could not fit {copula_name} copula -> {e}")
            continue

    if best_copula is not None:
        copula_results[t] = best_copula


###############################################################################
# 3) DISPLAY BEST COPULA PER ROLLING WINDOW
###############################################################################

print("\nBest Copula Models for Each Rolling Window:")
for t, (copula_name, copula) in copula_results.items():
    print(f"Window ending at {df_returns.index[t]}: {copula_name} Copula with AIC={best_aic:.2f}")


Fitting copulas: 100%|██████████| 400/400 [1:35:08<00:00, 14.27s/it]


Best Copula Models for Each Rolling Window:
Window ending at 2021-01-01 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-02 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-03 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-04 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-05 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-06 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-07 00:00:00: Gaussian Copula with AIC=-133832314806.09
Window ending at 2021-01-08 00:00:00: Student-t Copula with AIC=-133832314806.09
Window ending at 2021-01-09 00:00:00: Student-t Copula with AIC=-133832314806.09
Window ending at 2021-01-10 00:00:00: Student-t Copula with AIC=-133832314806.09
Window ending at 2021-01-11 00:00:00: Student-t Copula with AIC=-133832314806.09
Window ending at 2021-01-12 00:00:00: Student-t Copula with AIC=-133832

In [112]:
###############################################################################
# 5) FIT A STUDENT-T COPULA USING COPULAE ON THE FINAL STANDARDIZED RESIDUALS
###############################################################################

# Gather final standardized residuals from the last chosen model
final_std_resids = pd.DataFrame(index=df_returns.index, columns=df_returns.columns)
for asset in assets:
    best_fit = chosen_models.get((asset, last_day_idx), None)
    if best_fit is not None:
        final_std_resids[asset] = best_fit.resid / best_fit.conditional_volatility

final_std_resids = final_std_resids.dropna()

# Convert to [0,1] (pseudo-observations)
u_data = final_std_resids.apply(lambda col: to_uniform(col), axis=0).dropna()

# Fit a Student-t Copula
copula = StudentCopula(dim=u_data.shape[1])
copula.fit(u_data.values)

KeyboardInterrupt: 

In [70]:
###############################################################################
# 6) NEXT-DAY COVARIANCE FORECAST
###############################################################################

# Gather final forecasted volatilities for each asset
final_vols = []
for asset in assets:
    vol_val = vol_forecasts.loc[df_returns.index[last_day_idx], asset]
    # Fallback to historical standard deviation if forecast is missing
    if pd.isna(vol_val):
        vol_val = df_returns[asset].std()
    final_vols.append(vol_val)
final_vols = np.array(final_vols)

# Extract the correlation matrix from the fitted copula
copula_corr = copula.sigma

# Number of assets
n_assets = len(assets)

# Construct the forecasted covariance matrix
cov_forecast = np.zeros((n_assets, n_assets))
for i in range(n_assets):
    for j in range(n_assets):
        cov_forecast[i, j] = final_vols[i] * final_vols[j] * copula_corr[i, j]

# Convert to DataFrame for better readability
cov_forecast_df = pd.DataFrame(cov_forecast, index=assets, columns=assets)

# Display the forecasted covariance matrix
print("\n--- Next-Day Forecasted Covariance Matrix (Copula-GARCH) ---")
print(cov_forecast_df)



--- Next-Day Forecasted Covariance Matrix (Copula-GARCH) ---
               Tron    Solana  Avalanche       Xrp        Bnb    Bitcoin  \
Tron       0.034395  0.030008   0.040281  0.029656   0.224553   0.253639   
Solana     0.030008  0.192094   0.197769  0.109479   0.874951   1.048084   
Avalanche  0.040281  0.197769   0.342357  0.148943   1.189647   1.344949   
Xrp        0.029656  0.109479   0.148943  0.167503   0.766701   0.857114   
Bnb        0.224553  0.874951   1.189647  0.766701  11.935785   7.634084   
Bitcoin    0.253639  1.048084   1.344949  0.857114   7.634084  10.748240   
Doge       0.032388  0.144290   0.199109  0.128632   1.031704   1.168511   
Etherium   0.322170  1.236132   1.609053  1.036600   9.340192  10.620747   
PolkaDot   0.039279  0.156784   0.227855  0.134241   1.053903   1.122828   

               Doge   Etherium  PolkaDot  
Tron       0.032388   0.322170  0.039279  
Solana     0.144290   1.236132  0.156784  
Avalanche  0.199109   1.609053  0.227855  
Xrp  

In [51]:
###############################################################################
# 7) SCENARIO GENERATION
###############################################################################

num_scenarios = 5000

# 7a) Sample from the Student Copula in U-space
u_sim = copula.random(num_scenarios)  # shape: (5000, n_assets)

# 7b) Invert each dimension using the final chosen distribution from GARCH
simulated_returns = np.zeros((num_scenarios, n_assets))

for i, asset in enumerate(assets):
    best_fit = chosen_models.get((asset, last_day_idx), None)
    if best_fit is None:
        dist_name = 'normal'
        dof = 8.0
    else:
        dist_name = best_fit.distribution.name.lower()
        # Attempt to get dof from GARCH param
        dof = best_fit.params.get('nu', 8.0)

    vol_i = final_vols[i]
    u_col = u_sim[:, i]

    if 'student' in dist_name:
        z = t.ppf(u_col, df=dof)  # Student's t(\u03bd)
    elif 'normal' in dist_name:
        z = norm.ppf(u_col)       # standard normal
    elif 'skewt' in dist_name:
        # approximate skew-t by t
        z = t.ppf(u_col, df=dof)
    else:
        z = norm.ppf(u_col)
    
    simulated_returns[:, i] = z * vol_i

simulated_returns_df = pd.DataFrame(simulated_returns, columns=assets)
print("\nSimulated scenario returns (first 5 rows):")
print(simulated_returns_df.head())

print("\nFinished! Steps performed:")
print("1) Loaded prices from CSV, computed daily returns.")
print("2) Rolling GARCH(1,1) fit with distribution selection (normal, t, skewt).")
print("3) Fitted Student-t Copula (copulae) on standardized residuals.")
print("4) Constructed next-day covariance matrix (Copula correlation * GARCH vol).")
print("5) Generated scenario returns by sampling from the Student Copula.")

AttributeError: 'ARCHModelResult' object has no attribute 'distribution'